In [4]:
import pandas as pd
import numpy as np

In [5]:
# stack: 列から行へのピボット、積み上げるイメージ
# unstack: 行から列へのピボット、積み上げを列に逃がすイメージ
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [6]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [7]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [8]:
# unstackは最も深い階層を列に移動させるが、階層を指定することも出来る
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [9]:
# インデックス名でunstackする
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [10]:
# unstackは欠損値を発生させる場合がある
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [11]:
# stackは欠損値を埋める場合がある
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [12]:
# stackはdropna=Falseを指定することで欠損値を保持できる
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [14]:
df = pd.DataFrame({'left': result, 'right': result + 5}, columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [15]:
# DataFrameのunstack
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [16]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

In [19]:
# pivot
# longフォーマット: 縦に長い積み上げ型
# wideフォーマット: 横に長い
data = pd.read_csv('macrodata.csv')
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
data = pd.DataFrame(data.to_records(),
                 columns=pd.Index(['realgdp', 'infl', 'unemp'], name='item'),
                 index=periods.to_timestamp('D', 'end'))

ldata = data.stack().reset_index().rename(columns={0: 'value'})
wdata = ldata.pivot('date', 'item', 'value')

In [20]:
# dateとitemが主キーであることがわかる
ldata[:10]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340
5,1959-06-30,unemp,5.100
6,1959-09-30,realgdp,2775.488
7,1959-09-30,infl,2.740
8,1959-09-30,unemp,5.300
9,1959-12-31,realgdp,2785.204


In [21]:
# pivotメソッドに与えた最初の2つの引数は、行と列のインデックスに用いている列名を指定するもの
pivoted = ldata.pivot('date', 'item', 'value')
pivoted.head()

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


In [22]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,2.114669
1,1959-03-31,infl,0.000,-0.458137
2,1959-03-31,unemp,5.800,0.114374
3,1959-06-30,realgdp,2778.801,0.468192
4,1959-06-30,infl,2.340,0.177716
5,1959-06-30,unemp,5.100,0.156972
6,1959-09-30,realgdp,2775.488,1.183880
7,1959-09-30,infl,2.740,2.063251
8,1959-09-30,unemp,5.300,-0.913081
9,1959-12-31,realgdp,2785.204,-0.853991


In [23]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.458137  2.114669  0.114374
1959-06-30  2.34  2778.801   5.1  0.177716  0.468192  0.156972
1959-09-30  2.74  2775.488   5.3  2.063251  1.183880 -0.913081
1959-12-31  0.27  2785.204   5.6  0.241081 -0.853991 -0.833913
1960-03-31  2.31  2847.699   5.2  0.964970 -1.503813  0.294063

In [24]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


In [25]:
# pivotと同じ操作をset_indexとunstackで再現する
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.458137  2.114669  0.114374
1959-06-30  2.34  2778.801   5.1  0.177716  0.468192  0.156972
1959-09-30  2.74  2775.488   5.3  2.063251  1.183880 -0.913081
1959-12-31  0.27  2785.204   5.6  0.241081 -0.853991 -0.833913
1960-03-31  2.31  2847.699   5.2  0.964970 -1.503813  0.294063
1960-06-30  0.14  2834.390   5.2 -0.086760  1.065434 -0.499157
1960-09-30  2.70  2839.022   5.6  0.504766  0.085815  1.190621